In [12]:

from __future__ import print_function # so print doesn't show brackets

import numpy as np
import itertools as itr

import os as os
import sys as sys 
import pandas as pd
import warnings
import hashlib



sys.path.append(os.path.join("..", "Libraries","QML_lib"))
import Evo as evo
from QML import *
import ModelGeneration

global paulis_list
paulis_list = {'i' : evo.identity(), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}

   
"""
Functions for use by operator class to parse string (name) and prodcue relevent operators, lists etc.
"""

def print_matrix(name):
    op = operator(name)
    print(op.matrix)

def get_num_qubits(name):
    """
    Parse string and determine number of qubits this operator acts on. 
    """
    max_t_found = 0 
    t_str=''
    while name.count(t_str+'T')>0:
        t_str=t_str+'T'

    num_qubits = len(t_str) + 1
    return num_qubits
    

def list_used_qubits(name):
    """
    Parse string and determine which qubits are acted on non-trivially. 
    """
    max_t, t_str = find_max_letter(name, "T")
    max_p, p_str = find_max_letter(name, "P")
    running_list = []

    if max_p >= max_t:
        list_by_p_sep = []
        if p_str == '':  
          ## In case of empty separator, split by anything into one string    
          p_str = 'RRR'
        
        sep_by_p = name.split(p_str)
        for element in sep_by_p:
            list_by_p_sep.append(get_acted_on_qubits(element))

        for i in range(len(list_by_p_sep)):
            to_add= list(set(list_by_p_sep[i]) - set(running_list))
            running_list = running_list + to_add

    else:
        running_list = get_acted_on_qubits(name)
    return running_list


def get_acted_on_qubits(name):
    """
    Parse string and determine which qubits are acted on non-trivially. 
    """
    max_t, t_str = find_max_letter(name, "T")
    max_p, p_str = find_max_letter(name, "P")
    if max_p > max_t:
        list_by_p_sep = []
        if p_str == '':
          ## In case of empty separator, split by anything into one string    
          p_str = 'RRR'

        sep_by_p = name.split(p_str)
        for element in sep_by_p:
            list_by_sep.append(fill_qubits_acted_on_list, element)
    
    
    qubits_acted_on = []
    fill_qubits_acted_on_list(qubits_acted_on,name)
    return sorted(qubits_acted_on)
    
def fill_qubits_acted_on_list(qubits_acted_on, name):
    """
    Parse string and determine which qubits are acted on non-trivially. 
    Return list of those qubits. 
    """
    max_t, t_str = find_max_letter(name, "T")
    max_p, p_str = find_max_letter(name, "P")
    if(max_p > max_t):
        string_to_analyse = name.split(p_str)[0]
    else:
        string_to_analyse = name

    if max_t == 0:
        if string_to_analyse != 'i':
            qubits_acted_on.append(1)


    else:
        i=max_t
        this_t_str = t_str
        broken_down = string_to_analyse.split(this_t_str)
        lhs = broken_down[0]
        rhs = broken_down[1]
        if rhs !='i':
            qubits_acted_on.append(i+1)

        if max_t == 1:
            if lhs!='i':
                qubits_acted_on.append(1)
        else: 
            fill_qubits_acted_on_list(qubits_acted_on, lhs)                
    
def get_t_p_strings(name):
    """
    Find largest instance of consecutive P's and T's.
    Return those instances and lengths of those instances. 
    """
    t_str = ''
    p_str = ''
    while name.count(t_str+'T')>0:
        t_str=t_str+'T'

    while name.count(p_str+'P')>0:
        p_str=p_str+'P'

    max_t = len(t_str)
    max_p = len(p_str)

    return t_str, p_str, max_t, max_p        
    
def find_max_letter(string, letter):
    """
    Find largest instance of consecutive given 'letter'.
    Return largest instance and length of that instance. 
    """
    letter_str=''
    while string.count(letter_str+letter)>0:
        letter_str=letter_str+letter

    return len(letter_str), letter_str


def empty_array_of_same_dim(name):
    """
    Parse name to find size of system it acts on. 
    Produce an empty matrix of that dimension and return it. 
    """
    t_str=''
    while name.count(t_str+'T')>0:
        t_str=t_str+'T'

    num_qubits = len(t_str) +1
    dim = 2**num_qubits
    #print("String: ", name, " has NQubits: ", num_qubits)
    empty_mtx = np.zeros([dim, dim], dtype=np.complex128)
    return empty_mtx



def alph(name):
    """
    Return alphabetised version of name. 
    Parse string and recursively call alph function to alphabetise substrings. 
    """
    t_max, t_str = find_max_letter(name, "T")
    p_max, p_str = find_max_letter(name, "P")
    m_max, m_str = find_max_letter(name, "M")
    
    if p_max == 0 and t_max ==0 and p_max ==0 :
        return name
    
    if p_max > t_max and p_max > m_max: 
        ltr = 'P'
        string = p_str
    elif t_max >= p_max:
        string = t_str
        ltr = 'T'
    elif m_max >= p_max: 
        string = m_str
        ltr = 'M'
    elif t_max > m_max: 
        string = t_str
        ltr = 'T'
    else:
        ltr = 'M'
        string = m_str

    spread = name.split(string)
    if  p_max==m_max and p_max > t_max:
        string = p_str
        list_elements = name.split(p_str)
        
        for i in range(len(list_elements)):
            list_elements[i] = alph(list_elements[i])
        sorted_list = sorted(list_elements)
        linked_sorted_list = p_str.join(sorted_list)
        return linked_sorted_list
        
    if ltr=='P' and p_max==1:
        sorted_spread = sorted(spread)
        out = string.join(sorted_spread)
        return out
    elif ltr=='P' and p_max>1:
        list_elements = name.split(string)
        sorted_list = sorted(list_elements)
        for i in range(len(sorted_list)):
            sorted_list[i] = alph(sorted_list[i])
        linked_sorted_list = string.join(sorted_list)
        return linked_sorted_list
    else: 
        for i in range(len(spread)):
            spread[i] = alph(spread[i])
        out = string.join(spread)
        return out


def compute_t(inp):
    """
    Assuming largest instance of action on inp is tensor product, T.
    Parse string.
    Recursively call compute() function.
    Tensor product resulting lists.
    Return operator which is specified by inp.
    """
    max_t, t_str = find_max_letter(inp, "T")
    max_p, p_str = find_max_letter(inp, "P")

    if(max_p == 0 and max_t==0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 

    elif(max_t==0):
        return compute(inp)
    else:
        to_tens = inp.split(t_str)
        #print("To tens: ", to_tens)
        running_tens_prod=compute(to_tens[0])
        #print("Split by ", t_str, " : \n", to_tens)
        for i in range(1,len(to_tens)):
            max_p, p_str = find_max_letter(to_tens[i], "P")
            max_t, t_str = find_max_letter(to_tens[i], "T")
            #print("To tens [i=", i, "]:\n", to_tens[i] )
            rhs = compute(to_tens[i])
            running_tens_prod = np.kron(running_tens_prod, rhs)
        #print("RESULT ", t_str, " : ", inp, ": \n", running_tens_prod)
        return running_tens_prod

def compute_p(inp):
    """
    Assuming largest instance of action on inp is addition, P.
    Parse string.
    Recursively call compute() function.
    Sum resulting lists.
    Return operator which is specified by inp.
    """
    max_p, p_str = find_max_letter(inp, "P")
    max_t, t_str = find_max_letter(inp, "T")

    if(max_p == 0 and max_t==0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 

    elif max_p==0:
        return compute(inp)
    else: 
        to_add = inp.split(p_str)
        #print("To add : ", to_add)
        running_sum = empty_array_of_same_dim(to_add[0])
        for i in range(len(to_add)):
            max_p, p_str = find_max_letter(to_add[i], "P")
            max_t, t_str = find_max_letter(to_add[i], "T")

           # print("To add [i=", i, "]:", to_add[i] )
            rhs = compute(to_add[i])
            #print("SUM shape:", np.shape(running_sum))
            #print("RHS shape:", np.shape(rhs))
            running_sum += rhs

        #print("RESULT ", p_str, " : ", inp, ": \n", running_sum)
        return running_sum


def compute_m(inp):
    """
    Assuming largest instance of action on inp is multiplication, M.
    Parse string.
    Recursively call compute() function.
    Multiple resulting lists.
    Return operator which is specified by inp.
    """

    max_m, m_str = find_max_letter(inp, "M")
    max_p, p_str = find_max_letter(inp, "P")
    max_t, t_str = find_max_letter(inp, "T")

    if(max_m == 0 and max_t==0 and max_p == 0 ):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 

    elif max_m ==0:
        return compute(inp)
    
    else:   
        to_mult = inp.split(m_str)
        #print("To mult : ", to_mult)
        t_str=''
        while inp.count(t_str+'T')>0:
            t_str=t_str+'T'

        num_qubits = len(t_str) +1
        dim = 2**num_qubits

        running_product = np.eye(dim)

        for i in range(len(to_mult)):
            running_product = np.dot(running_product, compute(to_mult[i]))

        return running_product    
    
def compute(inp):
    """
    Parse string.
    Recursively call compute() functions (compute_t, compute_p, compute_m).
    Tensor product, multiply or sum resulting lists.
    Return operator which is specified by inp.
    """

    max_p, p_str = find_max_letter(inp, "P")
    max_t, t_str = find_max_letter(inp, "T")
    max_m, m_str = find_max_letter(inp, "M")

    if(max_m == 0 and max_t==0 and max_p == 0):
        pauli_symbol = inp
        return paulis_list[pauli_symbol] 
    elif max_m > max_t:
        return compute_m(inp)
    elif max_t >= max_p:
        return compute_t(inp)
    else:
        return compute_p(inp)    



In [13]:
class operator():
    """
    Operator class:
    Takes one argument: name (string) according to naming convention.
    Name specifies all details of operator. 
    e.g.
    - xPy is X+Y, 1 qubit
    - xTz is x TENSOR_PROD Z, 2 qubits
    - xMyTz is (X PROD Y) TENSOR_PROD Z, 2 qubits
    - xPzTiTTz is (X+Z) TENSOR_PROD I TENSOR_PROD Z
      -- 3 qubit operator. X+Z on qubit 1; I on qubit 2; Z on qubit 3
    See Naming_Convention.pdf for details.

    Constituents of an operator are operators of the same dimension which sum to give the operator.
    e.g. 
    - xPy = X + Y has constituents X, Y

    Assigns properties for : 
    - constituents_names: strings specifying constituents
    - constituents_operators: whole matrices of constituents
    - num_qubits: total dimension of operator [number of qubits it acts on]
    - matrix: total matrix operator
    - qubits_acted_on: list of qubits which are acted on non-trivially
      -- e.g. xTiTTz has list [1,3], since qubit 2 is acted on by identity
    - alph_name: rearranged version of name which follows alphabetical convention
      -- uniquely identifies equivalent operators for comparison against previously considered models
    -
    """
    def __init__(self, name, undimensionalised_name=None): 
        self.name = name
        if undimensionalised_name is not None:   
            self.undimensionalised_name = undimensionalised_name
    @property
    def constituents_names(self):
        """
        List of constituent operators names.
        """
        t_str, p_str, max_t, max_p = get_t_p_strings(self.name)
        paulis_list = {'i' : np.eye(2), 'x' : evo.sigmax(), 'y' : evo.sigmay(), 'z' : evo.sigmaz()}
        if(max_t >= max_p):
            # if more T's than P's in name, it has only one constituent. 
            return [self.name]
        else: 
            # More P's indicates a sum at the highest dimension. 
            return self.name.split(p_str)

    @property
    def num_qubits(self):
        """
        Number of qubits this operator acts on. 
        """
        return get_num_qubits(self.name)
        
    @property
    def constituents_operators(self):
        """
        List of matrices of constituents. 
        """
        ops = []
        for i in self.constituents_names:
            ops.append(compute(i))
        return ops

    @property
    def num_constituents(self):
        """
        Integer, how many constituents, and therefore parameters, are in this model.
        """    
        return len(self.constituents_names)
    
    @property 
    def matrix(self):
        """
        Full matrix of operator. 
        """
        mtx = empty_array_of_same_dim(self.name)
        for i in self.constituents_operators:
            mtx += i
        return mtx

    @property
    def qubits_acted_on(self):
        """
        List of qubits which are acted on non-trivially by this operator. 
        TODO: qubit count starts from 1 -- should it start from 0?
        """
        return list_used_qubits(self.name)
   
    @property 
    def two_to_power_used_qubits_sum(self):
        """
        Binary sum of operators acted on. 
        For use in comparing new operators. [Not currently used]
        """
        running_sum = 0
        for element in list_used_qubits(self.name):
            running_sum += 2**element
        return running_sum

    @property
    def alph_name(self):
        """
        Name of operator rearranged to conform with alphabetical naming convention. 
        Uniquely identifies equivalent operators. 
        For use when comparing potential new operators. 
        """
        return alph(self.name)
        

In [14]:
op = operator('x')

In [17]:
mt = op.matrix

In [18]:
from numpy import linalg as la

In [20]:
w,v=la.eig(mt)

In [21]:
w

array([ 1.+0.j, -1.+0.j])

In [22]:
v

array([[ 0.70710678-0.j,  0.70710678+0.j],
       [ 0.70710678+0.j, -0.70710678-0.j]])

In [32]:
a = np.sum(la.eig(mt)[1], axis=1)

In [34]:
a/np.linalg.norm(a)

array([  1.00000000e+00+0.j,   7.85046229e-17+0.j])

In [47]:
def ideal_probe(name):
    mtx = operator(name).matrix
    eigvalues = np.linalg.eig(mtx)[1]
    summed_eigvals = np.sum(eigvalues, axis=0)
    normalised_probe = summed_eigvals / np.linalg.norm(summed_eigvals)
    return normalised_probe

In [49]:
ideal_probe('y')

array([ 0.5-0.5j,  0.5-0.5j])

In [44]:
name = 'x'
mtx = operator(name).matrix
eigvalues = np.linalg.eig(mtx)[1]
summed_eigvals = np.sum(eigvalues, axis=0)
normalised_probe = summed_eigvals / np.norm(summed_eigvals)

AttributeError: module 'numpy' has no attribute 'norm'

In [43]:
eigvalues

array([[ 0.70710678-0.j,  0.70710678+0.j],
       [ 0.70710678+0.j, -0.70710678-0.j]])

In [ ]:
def compareModelsThroughIdealProbe(self, log_comparison_high=50.0, num_times_to_use = 'all', model_a_id = None, model_b_id =None, model_a = None, model_b = None, name_a=None, name_b=None, print_result = True):
    # Either pass in name_a and name_b OR model_a and model_b
    if model_a is None and model_b is None:
        if model_a_id is not None and model_b_id is not None: 
            model_a = self.getModelInstanceFromID(model_a_id)
            model_b = self.getModelInstanceFromID(model_b_id)
        else: # if only names were passed 
            model_a = self.getModelInstance(name_a)
            model_b = self.getModelInstance(name_b)
    if model_a ==  model_b:
        return "Same Models"
    else: 
        print("Computing Bayes Factor b/w ", model_a.Name, " & ", model_b.Name)
        log_comparison_low = 1.0/log_comparison_high
        if model_a_id is None and model_b is None:
            model_a_id = model_a.ModelID
            model_b_id = model_b.ModelID

        if num_times_to_use == 'all':
            times_a = model_a.TrackTime
        elif len(model_a.TrackTime) < num_times_to_use:
            times_a = model_a.TrackTime
        else: 
            times_a = model_a.TrackTime[num_times_to_use:]

        if num_times_to_use=='all':
            times_b = model_b.TrackTime
        elif len(model_b.TrackTime) < num_times_to_use:
            times_b = model_b.TrackTime
        else: 
            times_b = model_b.TrackTime[num_times_to_use:]
        gen_a = model_a.GenSimModel
        gen_b = model_b.GenSimModel

        # TODO: cutting times arrays because when one model finishes it doesn't have as many times as other. But it shouldn't be adding times together as list anyway??
#           num_times_to_use = min(len(times_a), len(times_b))
#            times = times_a[:num_times_to_use] + times_b[:num_times_to_use]

        times = []
        times.extend(times_a)
        times.extend(times_b)
        
        ideal_probe_a = model_a.Operator.ideal_probe
        ideal_probe_b = model_b.Operator.ideal_probe
        
        ideal_probes = [ideal_probe_a, ideal_probe_b]
        

        
        

In [ ]:
def updated_log_likelihood(model, times, ideal_probes):
    import copy
    updater = copy.deepcopy(model.Updater) #this could be what Updater.hypotheticalUpdate is for?
    updater.model.inBayesUpdates = True # updater.model is our gen
    updater.ideal_probes = ideal_probes
    
    for i in range(len(times)):
    
        exp = get_exp(model, gen, [times[i]])
        params_array = np.array([[model.FinalParams[0][0]]])
        datum = updater.model.simulate_experiment(params_array, exp)
        updater.update(datum, exp)

    log_likelihood = updater.log_total_likelihood
    del updater
    return log_likelihood        


In [ ]:
def BayesComparisonSameDimension(model_a, model_b, times):
    """ 
    Use ideal probes to try to distinguish between potentially locally equivalent models.
    """
    
    if model_a.log_total_likelihood > model_b.log_total_likelihood:
        ideal = model_a.Operator.ideal_probe
    else: 
        ideal = model_b.Operator.ideal_probe

    log_l_a = log_likelihood_same_dim(model_a, times, ideal)
    log_l_b = log_likelihood_same_dim(model_b, times, ideal)
    
    bayes_factor = np.exp(log_l_a - log_l_b)
    return bayes_factor
    
        
    